## Множественная проверка гипотез

**1.** Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

In [6]:
aucs = pd.read_csv('data/AUCs.txt', sep='\t', header=0)

In [7]:
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [8]:
corr_data = []

for i, lhs_column in enumerate(aucs.columns[1:]):
    for j, rhs_column in enumerate(aucs.columns[1:]):
        if i >= j:
            continue
        
        statistic, p = wilcoxon(aucs[lhs_column], aucs[rhs_column])
        corr_data.append([lhs_column, rhs_column, statistic, p])

In [9]:
aucs_correlation = pd.DataFrame.from_records(corr_data)
aucs_correlation.columns = ['AUC_A', 'AUC_B', 'statistic', 'p']
aucs_correlation

,AUC_A,AUC_B,statistic,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


**2.** Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [10]:
(aucs_correlation.p < 0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

**3.** Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [11]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [12]:
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject
aucs_correlation

,AUC_A,AUC_B,statistic,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [13]:
(aucs_correlation.p_corrected < 0.05).value_counts()

False    6
Name: p_corrected, dtype: int64

**4.** Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [15]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [16]:
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject
aucs_correlation

,AUC_A,AUC_B,statistic,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


In [17]:
(aucs_correlation.p_corrected < 0.05).value_counts()

True     3
False    3
Name: p_corrected, dtype: int64